We will introduce the following terms:

* containers
* iterables
* iterators
* generators

# Containers

Containers are actually any python data structure that fulfills the membership relationship with its objects, typical examples of such containers are lists/tuples/dicts:


In [1]:
pokemons = ['Bulbasaur',
            'Charmander',
            'Pikachu',
            'Mankey'] # Catch 'em all.
print('Bulbasaur' in pokemons)
print('Ponyta' not in pokemons)
print('bingorabbit' in pokemons)

True
True
False


* Containers are plain basic boxes, you can manage them (add/remove/edit their items - if possible -), that's why they are easy to deal with.
* Implementing a container requires implementing the **\_\_contains\_\_()** magic method.
* Containers provide a way to test if they have another object and they might provide a way to iterate over their contents (other contained objects) which would make them also iterables.

# Iterables

* In python, being able to iterate/loop over items in a container makes it iterable. So, if containers provide a way to iterate over their items - or get one of them when possible, else raises an **IndexError** - they become iterables.
* But an iterable is not by default a container (see [Bloom Filters](https://en.wikipedia.org/wiki/Bloom_filter).
* An iterable is something which we can iterate over to fetch data within, whether these data is file contents, socket stream.

In [2]:
pokedox = iter(pokemons)

In [3]:
for pokemon in pokedox:
    print("I'm an awesome pokemon and my name is {0}.".format(pokemon))

I'm an awesome pokemon and my name is Bulbasaur.
I'm an awesome pokemon and my name is Charmander.
I'm an awesome pokemon and my name is Pikachu.
I'm an awesome pokemon and my name is Mankey.


In [4]:
type(pokemons)

list

In [5]:
type(pokedox)

list_iterator

* As we can see, the pokemons list is an **iterable** but the pokedox object is an **iterator**.
* Implementing an iterator requires implementing the **\_\_iter\_\_()** magic method, which would return the iterator itself. Or the **\_\_getitem\_\_()** magic method to support the sequence protocol which calls for items in the container starting from index 0.

# Iterators

As per python docs, iterators are those items that support the [Iterator protocol](https://docs.python.org/3/library/stdtypes.html#iterator-types) - or as mentioned before, the sequence protocol, but we will focus on the first, which can be implemented by implementing the **\_\_iter\_\_()** and **next()**

In [6]:
class FibIterator(object):
    
    def __init__(self):
        self.previous, self.current = 0,1
    
    def __iter__(self):
        return self
   
    def __next__(self):
        value = self.current
        self.previous, self.current = self.current, self.current+self.previous
        return value

In [7]:
fibonacci = FibIterator()

In [8]:
print(next(fibonacci))
print(next(fibonacci))
print(next(fibonacci))

1
1
2


In [10]:
for i in range(10):
    print(next(fibonacci))

3
5
8
13
21
34
55
89
144
233


* Note that the iterator remembers where it stopped and continue from there, and once you reach the final item, it's done!
* https://docs.python.org/3/library/itertools.html

In [11]:
from itertools import count
counter = count(15)
for i in range(5):
    print(next(counter))

15
16
17
18
19


In [12]:
from itertools import cycle, islice

pokemons_cycle = cycle(pokemons) # This will create an infinite cycling iterator
pokemons_slice = islice(pokemons_cycle, 0, len(pokemons)*2) # Finite

for pokemon in pokemons_slice:
    print(pokemon)

Bulbasaur
Charmander
Pikachu
Mankey
Bulbasaur
Charmander
Pikachu
Mankey


# Generators
A generator is a more elegantly written iterator, so instead of implementing the Fib class up there and going thruogh implementing the **\_\_iter\_\_** and **\_\_next\_\_** methods, we can just do it like this:

In [57]:
def fib():
    prev, curr = 0, 1
    while True:
        yield curr
        prev, curr = curr, prev + curr

new_fib = fib()
for i in islice(new_fib, 0, 10):
    print(i)

1
1
2
3
5
8
13
21
34
55


* We introduce a new magic word, it's **yield**. Once it's there, we have a generator.
* The function **fib**, once called through the islice function, gets executed line by line, till it reaches the yield statement.
* Then it yields a value and stays idle in the memory, waiting us to call another value and another value ...
* When **islice** reaches its 10th element and we start to call the 11th, when it raises a StopIteration exception and exits.

# range() vs xrange()

In [13]:
print(range(5))

range(0, 5)


In [14]:
print(list(range(5)))

[0, 1, 2, 3, 4]


In [15]:
type(range(5))

range

In [16]:
type(list(range(5)))

list

# Appendix: How generators actually work

In [58]:
def echo(value=None):
     print("Execution starts when 'next()' is called for the first time.")
     try:
         while True:
             try:
                 value = (yield value)
             except Exception as e:
                 value = e
     finally:
         print("Don't forget to clean up when 'close()' is called.")

In [59]:
generator = echo(1)
print(next(generator))

Execution starts when 'next()' is called for the first time.
1


In [60]:
print(next(generator))

None


In [61]:
print(generator.send(2))

2


In [62]:
generator.close()

Don't forget to clean up when 'close()' is called.


### Use of generators

Generators boosts performance and memory usage, so the most simple usage the difference between **range** and **xrange**, the first returns a list, but the second returns a generator, so instead of occupying the memory with a list of numbers, it just give you the number you want when you need it. In python 3, range will return a genrator by default.

In [68]:
# Normal list
list_a = list(x * x for x in range(10))
type(list_a)
print([x * x for x in range(10)])

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [64]:
# List Comperhention
list_b = [x * x for x in range(10)]
type(list_b)

list

In [65]:
# Tuple Comperhention
tuple_a = (x * x for x in range(10)) # Should that be a tuple, ha? :+1:
type(tuple_a)

generator